In [1]:
#import dependencies
import pandas as pd


In [ ]:
# Read the CSV files
movies_metadata_csv = pd.read_csv("src/movies_metadata.csv", low_memory=False)
credits_part1_csv = pd.read_csv("src/credits_part1.csv", low_memory=False)
credits_part2_csv = pd.read_csv("src/credits_part2.csv", low_memory=False)

# Combine the two credits CSVs into a single DataFrame, then drop duplicates ids
credits_csv = pd.concat([credits_part1_csv, credits_part2_csv])
credits_csv = credits_csv.drop_duplicates(subset='id')

#convert ids to numeric
movies_metadata_csv["id"] = pd.to_numeric(movies_metadata_csv["id"], errors="coerce")

#drop nan ids & remove duplicate ids
movies_metadata_csv = movies_metadata_csv.dropna(subset=["id"]).drop_duplicates(subset='id')

# Merge movies metadata with credits on 'id' & convert id to int & Set 'id' as the index
merged_df = pd.merge(movies_metadata_csv, credits_csv, on='id', how='left').astype({'id': 'int'}).set_index('id')

#english only
merged_df = merged_df[merged_df["original_language"] == "en"]

#released on or after 1980-01-01
merged_df = merged_df[merged_df["release_date"] >= "1980-01-01"]



    
#display
merged_df